In [1]:
'''
To apply the previous simulations to multiple instrument portfolios, all that has to be added is accounting
for correlation between risk factors and positions of each security

To accomplish this, model a derivative position and a portfolio of derivatives
'''


'\nTo apply the previous simulations to multiple instrument portfolios, all that has to be added is accounting\nfor correlation between risk factors and positions of each security\n\nTo accomplish this, model a derivative position and a portfolio of derivatives\n'

In [2]:
# Imports from previous chapters
class market_environment(object):
    '''
    class to model a market environment relevant for valuation
    
    Attributes
    ===========
    name: string - name of the market environment
    pricing_date: datetime - date of the market environment
    
    Methods
    =========
    add_constant: adds a constant market parameter
    get_constant: get constant
    add_lists: adds a list (of underlyings)
    get_list: gets a list
    add_curve: adds a market curve (i.e. yield curve)
    get_curve: gets a market curve
    add_environment: upserts whole market environments with constants, lists, curves
    
    '''
    def __init__(self, name, pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self, key, constant):
        self.constants[key] = constant
    
    def get_constant(self, key):
        return self.constants[key]
    
    def add_list(self, key, list_object):
        self.lists[key] = list_object
        
    def get_list(self, key):
        return self.lists[key]
    
    def add_curve(self, key, curve):
        self.curves[key] = curve
        
    def get_curve(self, key):
        return self.curves[key]
    
    def add_environment(self, env):
        # overwrite values for class if they exist
        self.constants.update(env.constants)
        self.lists.update(env.lists)
        self.curves.update(env.curves)
        

In [3]:
'''
Lastly, let's do Square-Root Diffusion, like Cox, Ingersoll, Ross did for stochastic short term rates
'''
class square_root_diffusion(simulation_class):
    ''' Class to generate simulated paths based on the Cox-Ingersoll-Ross (1985) square-root diffusion model.
    Attributes
    =========
    name: string - name of the object
    mar_env: instance of a market environment
    corr: Boolean - True if correlated with an object
    
    Methods
    =========
    update: updates parameters
    generate_paths: return the Monte Carlo paths for this model
    '''
    
    def __init__(self, name, mar_env, corr=False):
        super(square_root_diffusion, self).__init__(name, mar_env, corr)
        self.kappa = mar_env.get_constant('kappa') # how fast does rate revert to mean?
        self.theta = mar_env.get_constant('theta') # long sense stationary mean
        
    def update(self, initial_value=None, volatility=None, kappa=None, theta=None, final_date=None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if kappa is not None:
            self.kappa = kappa
        if theta is not None:
            self.theta = theta
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None
        
    def generate_paths(self, fixed_seed=True, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        M = len(self.time_grid)
        I = self.paths
        paths = np.zeros((M, I))
        paths_ = np.zeros_like(paths)
        paths[0] = self.initial_value
        paths_[0] = self.initial_value

        if self.correlated is False:            
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            rand = self.random_numbers
            
        for t in range(1, len(self.time_grid)):
            dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
            if self.correlated is False:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = rand[self.rn_set]
                
            # full truncation Euler discretization
            paths_[t] = (paths_[t-1] + self.kappa *
                        (self.theta - np.maximum(0, paths_[t-1, :])) * dt + 
                        self.volatility * np.sqrt(dt) * ran)
            paths[t] = np.maximum(0, paths_[t]) # 0 of Euler discretization step
        self.instrument_values = paths

NameError: name 'simulation_class' is not defined

In [4]:
class jump_diffusion(simulation_class):
        '''
        Now let's add the jump diffusion term to it from the Merton76 model
        
        Attributes
        =========
        name: str - name of the object
        mar_env: market_environment instance
        corr: bool - true if model objects are correlated
        
        Methods
        =========
        update: updates parameters
        generate_paths: returns a Monte Carlo of jump diffusion model
        ''' 
        def __init__(self, name, mar_env, corr=False):
            super(jump_diffusion, self).__init__(name, mar_env, corr)
            self.lamb = mar_env.get_constant('lambda')
            self.mu = mar_env.get_constant('mu')
            self.delt = mar_env.get_constant('delta')
            print(self)
            
        def update(self, initial_value=None, volatility=None, lamb=None, mu=None, delta=None, final_date=None):
            if initial_value is not None:
                self.initial_value = initial_value
            if volatility is not None:
                self.volatility = volatility
            if lamb is not None:
                self.lamb = lamb
            if mu is not None:
                self.mu = mu
            if delta is not None:
                self.delt = delta
            if final_date is not None:
                self.final_date = final_date
            self.instrument_values = None
            
        def generate_paths(self, fixed_seed=False, day_count=365.):
            print(self)
            if self.time_grid is None:
                self.generate_time_grid()
            # num dates
            M = len(self.time_grid)
            # num paths
            I = self.paths
            
            paths = np.zeros((M, I))
            # init simulation paths to initial security price
            paths[0] = self.initial_value
            
            if self.correlated is False:
                sn1 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)

            else:
                # If correlated, take the (correlated) random numbers from the market environment
                sn1 = self.random_numbers
                
            # now the jump compent distribution (assuming no correlation between security path and jumps)
            sn2 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
            
            rj = self.lamb * (np.exp(self.mu + 0.5 * self.delt ** 2) - 1)
            
            short_rate = self.discount_curve.short_rate
            
                
            for t in range(1, len(self.time_grid)):
                if self.correlated is False:
                    ran = sn1[t]
                else:
                    # select (correlated) random number by Cholesky dot product
                    # second term is sn1 from 0, to t, taking all
                    ran = np.dot(self.cholesky_matrix, sn1[:, t, :])
                    # replace the non correlated ones with the correlated ones
                    ran = ran[self.rn_set]

                # Discretization time slice
                dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
                
                # Poisson-distributed random numbers for the jump component term
                poi = np.random.poisson(self.lamb * dt, I)
                print(poi[:10])
                # Euler Discretization time - note that last term is either 0 or a jump based
                # on Poisson result for time t
                paths[t] = paths[t - 1] * (
                np.exp((short_rate - rj -
                        0.5 * self.volatility ** 2) * dt +
                       self.volatility * np.sqrt(dt) * ran) +
                (np.exp(self.mu + self.delt * sn2[t]) - 1) * poi)
                
            self.instrument_values = paths
                    
        

NameError: name 'simulation_class' is not defined

In [5]:
'''
This is useful for calculating the general discount factor, in this case
exp(-r * t) at time t
Modeling this as a class:
'''
class constant_short_rate(object):
    '''
    Class for constant short rate discounting.
    
    Attributes
    ===========
    name: string - name of the object
    short_rate: float (positive) - constant rate for discounting
    
    Results
    ==========
    get_discount_factors: get discount factors given a list/array of timestamp objects
    '''
    
    def __init__(self, name, short_rate):
        self.name = name
        self.short_rate = short_rate
        if(short_rate < 0):
            raise ValueError('Short rate negative.')
    
    def get_discount_factors(self, date_list, dtobjects=True):
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * np.sort(-dlist)) # take negative of time fractions
        return np.array((date_list, dflist)).T

In [6]:
'''
Now build from a generic simulation class towards specific applications

'''
import numpy as np
import pandas as pd
class simulation_class(object):
    '''
    Provide base methods for simulation classes
    
    Params
    =========
    name: str - name of the object
    mar_env: market_environment - a market environment
    corr: bool - is this object correlated with another modeled object?
    
    Methods
    =========
    generated_time_grid: returns discretized time intervals for simulation
    get_instrument_values: returns current values of security simulated (as ndarray)
    '''
    
    def __init__(self, name, mar_env, corr):
        self.name = name
        self.pricing_date = mar_env.pricing_date
        self.initial_value = mar_env.get_constant('initial_value')
        self.volatility = mar_env.get_constant('volatility')
        self.final_date = mar_env.get_constant('final_date')
        self.currency = mar_env.get_constant('currency')
        self.frequency = mar_env.get_constant('frequency')
        self.paths = mar_env.get_constant('paths')
        self.discount_curve = mar_env.get_curve('discount_curve')
        try:
            # If there is a time grid in mar_env take it
            self.time_grid = mar_env.get_list('time_grid')
        except:
            self.time_grid = None
        try:
            # if there are special dates, add those
            self.special_dates = mar_env.get_list('special_dates')
        except:
            self.special_dates = []
        self.instrument_values = None
        self.correlated = corr
        if corr is True:
            # only needed when risk factors are correlated
            self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
            self.rn_set = mar_env.get_list('rn_set')[self.name]
            self.random_numbers = mar_env.get_list('random_numbers')
        
    def generate_time_grid(self):
        start = self.pricing_date
        end = self.final_date
        # use pandas date range func to generate time grid
        # freq is pandas frequency char
        time_grid = pd.date_range(start=start, end=end, freq=self.frequency).to_pydatetime()
        time_grid = list(time_grid)
        # add begin, end, special
        if start not in time_grid:
            time_grid.insert(0, start)
            
        if end not in time_grid:
            time_grid.append(end)
        if len(self.special_dates) > 0:
            # add all
            time_grid.extend(self.special_dates)
            # de-dup
            time_grid = list(set(time_grid))
            # sort
            time_grid.sort()
        # cast back to np array
        self.time_grid = np.array(time_grid)
    
    def get_instrument_values(self, fixed_seed=True):
        if self.instrument_values is None:
            # only initiative simulation if no values
            self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
        elif fixed_seed is False:
            # also resim if not a fixed seed (i.e. monte carlo)
            self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
        # Otherwise all set so return what was simulated already
        return self.instrument_values

In [7]:
'''
Set up the Euler Discretizatin of the Geometric Brownian Motion PDE
'''
class geometric_brownian_motion(simulation_class):
    '''
    Class to generate simulated paths based on the Black-Scholes-Merton GBM model
    
    Attributes
    =========
    name: string - name of obj
    mar_env: market_environment instance
    corr: Boolean - is this correlated to other modeled objects?
    
    Methods
    =========
    update: updates parameters
    generate_paths: returns Monte Carlo paths given env
    '''
    def __init__(self, name, mar_env, corr=False):
        super(geometric_brownian_motion, self).__init__(name, mar_env, corr)
        
    def update(self, initial_value=None, volatility=None, final_date=None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None
        
    def generate_paths(self, fixed_seed=False, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        # Grid size
        M = len(self.time_grid)
        # Number of Paths for Monte
        I = self.paths
        # ndArray of the Monte Carlo shape (time steps by number of evolutions)
        paths = np.zeros((M, I))
        # initialize first step of each trial with initial value
        paths[0] = self.initial_value

        if not self.correlated:
            # if not correlated, go random
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            # if correlated, use correlation random numbers object from market_env
            rand = self.random_numbers

        short_rate = self.discount_curve.short_rate
        # short rate for drift process
        
        for t in range(1, len(self.time_grid)):
            # select right idx from random number set
            if not self.correlated:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
        
            # difference between two dates as a year fraction (infinitisemal for Euler discretization)
            dt = (self.time_grid[t]- self.time_grid[t-1]).days / day_count        

            paths[t] = paths[t-1] * np.exp((short_rate - 0.5 * self.volatility ** 2) * dt +
                                          self.volatility * np.sqrt(dt) * ran)
            # ran is Brownian motion evolution
            # dt is time slice (weighting for this slice's evolution)
        
        self.instrument_values = paths
            

In [8]:
class valuation_class(object):
    '''
    Basic class for single factor derivative valuation
    Attributes
    =========
    name: str - name of the object
    underlying: instance of a simulation class modeling single risk factor
    mar_env: instance of market environment
    payoff_func: str - derivative payoff as a python function that can be evaluated
    
    Methods
    =========
    update: updates selected valuation parameters
    delta: returns delta of derivative (change in price to underlying risk factor)
    vega: returns vega of derivative (change in price to implied volatility)
    '''
    def __init__(self, name, underlying, mar_env, payoff_func=''):
        self.name = name
        self.pricing_date = mar_env.pricing_date
        try:
            # strike price is optional
            self.strike = mar_env.get_constant('strike')
        except:
            pass
        self.maturity = mar_env.get_constant('maturity')
        self.currency = mar_env.get_constant('currency')
        # simulation parameters and discount curve from simulation object
        self.frequency = underlying.frequency
        self.paths = underlying.paths
        self.discount_curve = underlying.discount_curve
        self.payoff_func = payoff_func
        self.underlying = underlying
        # provide pricing and maturity date to underlying
        self.underlying.special_dates.extend([self.pricing_date, self.maturity])
        
    def update(self, initial_value=None, volatility=None, strike=None, maturity=None):
        if initial_value is not None:
            self.underlying.update(initial_value=initial_value)
        if volatility is not None:
            self.underlying.update(volatility=volatility)
        if strike is not None:
            self.strike = strike
        if maturity is not None:
            self.maturity = maturity
            # add new maturity date if not already in time_grid
            if maturity not in self.underlying.time_grid:
                self.underlying.special_dates.append(maturity)
                self.underlying.instrument_values = None # because this means it is first update
                
    def delta(self, interval=None, accuracy=4):
        if interval is None:
            interval = self.underlying.initial_value / 50. # ?
        # finite difference spline approximation of Delta
        # f(a)
        value_left = self.present_value(fixed_seed=True)
        # first create a t+1 value
        initial_del = self.underlying.initial_value + interval
        self.underlying.update(initial_value=initial_del)
        # now take f(b)
        value_right = self.present_value(fixed_seed=True)
        # reset initial value of security under simulation
        self.underlying.update(initial_value = initial_del - interval)
        # finite difference
        delta = (value_right - value_left) / interval
        # correct for potential floating point errors pushing outside bounds of Delta value
        if delta < -1.0:
            return -1.0
        elif delta > 1.0:
            return 1.0
        else:
            return round(delta, accuracy)
        
    def vega(self, interval=0.01, accuracy=4):
        if interval < self.underlying.volatility / 50.:
            interval = self.underlying.volatility / 50.
        # finite difference again
        value_left = self.present_value(fixed_seed=True)
        # increment by epsilon amount
        vola_del = self.underlying.volatility + interval
        # update simulation object
        self.underlying.update(volatility=vola_del)
        # f(a + ∆)
        value_right = self.present_value(fixed_seed=True)
        # reset simulation security volatility value
        self.underlying.update(volatility = vola_del - interval)
        vega = (value_right - value_left) / interval
        return round(vega, accuracy)   

In [9]:
'''
The continuation value of an American option at time t is the maximum of the value of exercising or continuing.
An optimal stopping problem

The continuation value is approximated by an OLS regression with parameters estimated by fitting regression to the
actual continuation value
''' 
class valuation_mcs_american(valuation_class):
    '''
    Class to value American options with arbitrary payoff by single-factor Monte Carlo simulation
    
    Methods
    =========
    generate_payoff: returns payoffs given the paths and payoff functions

    present_value: returns present value (LSM Monte Carlo estimator)
    '''
    def generate_payoff(self, fixed_seed=False):
        '''
        Parameters
        =========
        fixed_seed: use same/fixed seed for valuation
        '''
        try:
            # strike optional
            strike = self.strike
        except:
            print("No strike specified")
            pass
        
        paths = self.underlying.get_instrument_values(fixed_seed=fixed_seed)
        time_grid = self.underlying.time_grid
        time_index_start = int(np.where(time_grid == self.pricing_date)[0])
        time_index_end = int(np.where(time_grid == self.maturity)[0])
        instrument_values = paths[time_index_start:time_index_end + 1] # value of underlying
        payoff = eval(payoff_func)
        return instrument_values, payoff, time_index_start, time_index_end
    
    def present_value(self, accuracy=6, fixed_seed=False, bf=5, full=False):
        '''
        Parameters
        =========
        accuracy: int - fixed point precision
        fixed_seed: bool - fixed seed or no
        bf: int - number of basis functions for regression
        full: bool - also return full 1d array of present values
        '''
        instrument_values, inner_values, time_index_start, time_index_end = \
            self.generate_payoff(fixed_seed=fixed_seed)
        time_list = self.underlying.time_grid[time_index_start:time_index_end + 1]
        discount_factors = self.discount_curve.get_discount_factors(time_list, dtobjects=True)
        V = inner_values[-1] # inner value at end of time
        for t in range(len(time_list) - 2, 0, -1): # work backwards
            # discount factor for epsilon slice
            df = discount_factors[t, 1] / discount_factors[t + 1, 1]
            # regress
            # fit instrument value to discounted inner value at end of time with polynominal order bf
            rg = np.polyfit(instrument_values[t], V * df, bf)
            # calculate continuation value per path
            C = np.polyval(rg, instrument_values[t])
            # Optimal decision - inner value if higher than continuation, otherwise continuation
            V = np.where(inner_values[t] > C, inner_values[t], V*df)
        df = discount_factors[0, 1] / discount_factors[1,1]
        result = df * np.sum(V) / len(V) # Discount all results
        if full:
            return round(result, accuracy), df * V # return result and discounted end value
        else:
            return round(result, accuracy) # just return result

In [24]:
'''
Lastly, let's do Square-Root Diffusion, like Cox, Ingersoll, Ross did for stochastic short term rates
'''
class square_root_diffusion(simulation_class):
    ''' Class to generate simulated paths based on the Cox-Ingersoll-Ross (1985) square-root diffusion model.
    Attributes
    =========
    name: string - name of the object
    mar_env: instance of a market environment
    corr: Boolean - True if correlated with an object
    
    Methods
    =========
    update: updates parameters
    generate_paths: return the Monte Carlo paths for this model
    '''
    
    def __init__(self, name, mar_env, corr=False):
        super(square_root_diffusion, self).__init__(name, mar_env, corr)
        self.kappa = mar_env.get_constant('kappa') # how fast does rate revert to mean?
        self.theta = mar_env.get_constant('theta') # long sense stationary mean
        
    def update(self, initial_value=None, volatility=None, kappa=None, theta=None, final_date=None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if kappa is not None:
            self.kappa = kappa
        if theta is not None:
            self.theta = theta
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None
        
    def generate_paths(self, fixed_seed=True, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        M = len(self.time_grid)
        I = self.paths
        paths = np.zeros((M, I))
        paths_ = np.zeros_like(paths)
        paths[0] = self.initial_value
        paths_[0] = self.initial_value

        if self.correlated is False:            
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            rand = self.random_numbers
            
        for t in range(1, len(self.time_grid)):
            dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
            if self.correlated is False:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = rand[self.rn_set]
                
            # full truncation Euler discretization
            paths_[t] = (paths_[t-1] + self.kappa *
                        (self.theta - np.maximum(0, paths_[t-1, :])) * dt + 
                        self.volatility * np.sqrt(dt) * ran)
            paths[t] = np.maximum(0, paths_[t]) # 0 of Euler discretization step
        self.instrument_values = paths

In [10]:
class valuation_mcs_european(valuation_class):
    '''
    Class to value European Options by single-factor Monte Carlo simulation.
    
    Methods
    =========
    generate_payoff: returns payoff given paths and given payoff function
    present_value: returns present value (Monte Carlo estimator)
    '''
    def generate_payoff(self, fixed_seed=False):
        '''
        Parameters
        =========
        fixed_seed: bool - use a static or random seed for valuation
        '''
        try:
            # strike optional
            strike = self.strike
        except:
            print("Missing strike!")
            pass
        paths = self.underlying.get_instrument_values(fixed_seed=fixed_seed)
        time_grid = self.underlying.time_grid
        try:
            time_index = np.where(time_grid == self.maturity)[0] # should be a one member vector
            time_index = int(time_index) # cast
        except:
            print("Maturity date not in the grid of underlying")
        maturity_value = paths[time_index] # price at expiry
        # average value over full path
        mean_value = np.mean(paths[:time_index], axis=1)
        # max val
        max_value = np.amax(paths[:time_index], axis=1)[-1]
        # min val
        min_value = np.amin(paths[:time_index], axis=1)[-1]
        try:
            payoff = eval(self.payoff_func)
            return payoff
        except:
            print("Error evaluating payoff function")
        
    def present_value(self, accuracy=6, fixed_seed=False, full=False):
        '''
        Parameters
        =========
        accuracy: int - fixed point precision
        fixed_seed: bool - use a static or random seed for valuation
        full: bool - also return full vector of present values
        '''
        cash_flow = self.generate_payoff(fixed_seed=fixed_seed)
        # Exponential function discount factor for lifetime of underlying
        discount_factor = self.discount_curve.get_discount_factors((self.pricing_date, self.maturity))[0][1]
        # Risk-neutral simple average of payoff profile 
        result = discount_factor * np.sum(cash_flow) / len(cash_flow)
        if full:
            return round(result, accuracy), discount_factor * cash_flow #?
        else:
            return round(result, accuracy)

In [22]:
class jump_diffusion(simulation_class):
        '''
        Now let's add the jump diffusion term to it from the Merton76 model
        
        Attributes
        =========
        name: str - name of the object
        mar_env: market_environment instance
        corr: bool - true if model objects are correlated
        
        Methods
        =========
        update: updates parameters
        generate_paths: returns a Monte Carlo of jump diffusion model
        ''' 
        def __init__(self, name, mar_env, corr=False):
            super(jump_diffusion, self).__init__(name, mar_env, corr)
            self.lamb = mar_env.get_constant('lambda')
            self.mu = mar_env.get_constant('mu')
            self.delt = mar_env.get_constant('delta')
            print(self)
            
        def update(self, initial_value=None, volatility=None, lamb=None, mu=None, delta=None, final_date=None):
            if initial_value is not None:
                self.initial_value = initial_value
            if volatility is not None:
                self.volatility = volatility
            if lamb is not None:
                self.lamb = lamb
            if mu is not None:
                self.mu = mu
            if delta is not None:
                self.delt = delta
            if final_date is not None:
                self.final_date = final_date
            self.instrument_values = None
            
        def generate_paths(self, fixed_seed=False, day_count=365.):
            print(self)
            if self.time_grid is None:
                self.generate_time_grid()
            # num dates
            M = len(self.time_grid)
            # num paths
            I = self.paths
            
            paths = np.zeros((M, I))
            # init simulation paths to initial security price
            paths[0] = self.initial_value
            
            if self.correlated is False:
                sn1 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)

            else:
                # If correlated, take the (correlated) random numbers from the market environment
                sn1 = self.random_numbers
                
            # now the jump compent distribution (assuming no correlation between security path and jumps)
            sn2 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
            
            rj = self.lamb * (np.exp(self.mu + 0.5 * self.delt ** 2) - 1)
            
            short_rate = self.discount_curve.short_rate
            
                
            for t in range(1, len(self.time_grid)):
                if self.correlated is False:
                    ran = sn1[t]
                else:
                    # select (correlated) random number by Cholesky dot product
                    # second term is sn1 from 0, to t, taking all
                    ran = np.dot(self.cholesky_matrix, sn1[:, t, :])
                    # replace the non correlated ones with the correlated ones
                    ran = ran[self.rn_set]

                # Discretization time slice
                dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
                
                # Poisson-distributed random numbers for the jump component term
                poi = np.random.poisson(self.lamb * dt, I)
                print(poi[:10])
                # Euler Discretization time - note that last term is either 0 or a jump based
                # on Poisson result for time t
                paths[t] = paths[t - 1] * (
                np.exp((short_rate - rj -
                        0.5 * self.volatility ** 2) * dt +
                       self.volatility * np.sqrt(dt) * ran) +
                (np.exp(self.mu + self.delt * sn2[t]) - 1) * poi)
                
            self.instrument_values = paths
                    
        

In [11]:
class derivatives_position(object):
    '''
    Class to model a derivatives position
    
    Attributes
    =========
    name: str - name of object
    quantity: float - number of assets making up position
    underlying: string - security derivative is on
    mar_env: market environment instance
    otype: str - valuation class to use
    payoff_func: str - Python payoff func to use (uses eval())
    
    Methods
    =========
    get_info: Prints information about this position
    '''
    def __init__(self, name, quantity, underlying, mar_env, otype, payoff_func):
        self.name = name
        self.quantity = quantity
        self.underlying = underlying
        self.mar_env = mar_env
        self.otype = otype
        self.payoff_func = payoff_func
        
    def get_info(self):
        print('NAME')
        print(self.name, '\n')
        print('QUANTITY')
        print(self.quantity, '\n')
        print('Market Environment')
        print('\n**Constants**')
        for key, value in self.mar_env.constants.items():
            print(key, value)
        print('\n**Lists**')
        for key, value in self.mar_env.lists.items():
            print(key, value)
        print('\n**Curves')
        for key, value in self.mar_env.constants.items():
            print(key, value)
        print('\nOPTION TYPE')
        print(self.otype, '\n')
        print('PAYOFF FUNCTION')
        print(self.payoff_func)

In [12]:
import datetime as dt
me_gbm = market_environment('me_gbm', dt.datetime(2020, 1, 1))

In [13]:
me_gbm.add_constant('initial_value', 36.)
me_gbm.add_constant('volatility', 0.2)
me_gbm.add_constant('currency', 'EUR')

In [14]:
me_gbm.add_constant('model', 'gbm')

In [15]:
me_am_put = market_environment('me_am_put', dt.datetime(2020, 1, 1))

In [16]:
me_am_put.add_constant('maturity', dt.datetime(2020, 12, 31))
me_am_put.add_constant('strike', 40.)
me_am_put.add_constant('currency', 'EUR')

In [17]:
payoff_func = 'np.maximum(strike - instrument_values, 0)'

In [18]:
am_put_pos = derivatives_position(name='am_put_position',
                                 quantity=3,
                                 underlying='gbm',
                                 mar_env=me_am_put,
                                 otype='American',
                                 payoff_func=payoff_func)

In [19]:
am_put_pos.get_info()

NAME
am_put_position 

QUANTITY
3 

Market Environment

**Constants**
maturity 2020-12-31 00:00:00
strike 40.0
currency EUR

**Lists**

**Curves
maturity 2020-12-31 00:00:00
strike 40.0
currency EUR

OPTION TYPE
American 

PAYOFF FUNCTION
np.maximum(strike - instrument_values, 0)


In [20]:
'''
A "Relevant Market" contains all relevant risk factors and their correlations, along with the derivative positions
contemplated. 
'''

'\nA "Relevant Market" contains all relevant risk factors and their correlations, along with the derivative positions\ncontemplated. \n'

In [41]:
import pandas as pd
# Derivatives portfolio class

# Available Models
models = {'gbm': geometric_brownian_motion,
         'jd': jump_diffusion,
         'srd': square_root_diffusion}
# exercise types
otypes = {'European': valuation_mcs_european,
         'American': valuation_mcs_american}

class derivatives_portfolio(object):
    '''
    Class for modeling and valuing portfolios of derivatives
    
    Attributes
    =========
    name: str - name of object
    positions: dict - dictionary of positions (which are instances of derivatives_position)
    val_env: market_environment - market environment for valuation
    assets: dict - dictionary of market environments for assets
    correlations: list - correlations between assets
    fixed_seed: bool - fixed seed or random for simulations
    
    Methods
    =========
    get_positions: prints information about single portfolio positions
    get_statistics: returns a pandas Dataframe with portfolio stats
    '''
    def __init__(self, name, positions, val_env, assets, correlations=None, fixed_seed=False):
        self.name = name
        self.positions = positions
        self.val_env = val_env
        self.assets = assets
        self.underlyings = set()
        self.correlations = correlations
        self.time_grid = None
        self.underlying_objects = {}
        self.valuation_objects = {}
        self.fixed_seed = fixed_seed
        self.special_dates = []
        for pos in self.positions:
            # determine earliest starting date in portfolio
            self.val_env.constants['starting_date'] = min(self.val_env.constants['starting_date'],
                                                         positions[pos].mar_env.pricing_date)
            # latest date of position maturity or final market environment date
            self.val_env.constants['final_date'] = max[self.val_env.constants['final_date'], 
                                                       positions[pos].mar_env.constants['maturity']]
            # collect all underlyings and add to set (avoids redunancies)
            self.underlyings.add(positions[pos].underlying)
            
        # Generate time grid for full portfolio
        start = self.val_env.constants['starting_date']
        end = self.val_env.constants['final_date']
        time_grid = pd.date_range(start=start, end=end, freq=self.val_env.constants['frequency']).to_pydatetime()
        
        time_grid = list(time_grid)
        for pos in self.positions:
            maturity_date = positions[pos].mar_env.constants['maturity']
            if maturity_date not in time_grid:
                time_grid.insert(0, maturity_date)
                self.special_dates.append(maturity_date)
        if start not in time_grid:
            time_grid.insert(0, start)
        if end not in time_grid:
            time_grid.append(end)
        # remove duplicates
        time_grid = list(set(time_grid))
        # sort grid
        time_grid.sort()
        self.time_grid = np.array(time_grid)
        self.val_env.add_list('time_grid', time_grid)
        
        if correlations is not None:
            # deal with correlation list
            ul_list = sorted(self.underlyings)
            correlation_matrix = np.zeros(len(ul_list), len(ul_list))
            np.fill_diagonal(correlation_matrix, 1.0) # perfect self-correlation of course
            correlation_matrix = pd.DataFrame(correlation_matrix, index=ul_list, columns=ul_list)
            
            for i,j, corr in correlations:
                corr = min(corr, 0.99999999999)
                # RC fill in
                correlation_matrix.loc[i,j] = corr
                correlation_matrix.loc[j,i] = corr # why not do this as a triangular matrix instead?
                
            # Cholesky
            cholesky_matrix = np.linalg.cholesky(np.array(correlation_matrix))
            
            # Make a dictionary of indices of random number array used for stochastic term of underlying
            rn_set = {asset: ul_list.index(asset) for asset in self.underlyings}
            
            # Use these slices for all underlyings (allows correlations to be applied)
            random_numbers = sn_random_numbers((len(rn_set),
                                              len(self.time_grid),
                                              self.val_env.constants['paths']),
                                             fixed_seed=self.fixed_seed)
            
            # put all of this into the environment
            self.val_env.add_list('cholesky_matrix', cholesky_matrix)
            self.val_env.add_list('random_numbers', random_numbers)
            self.val_env.add_list('rn_set', rn_set)
            
        for asset in self.underlyings:
            # select market env
            mar_env = self.assets[asset]
            # add val environment to market env
            mar_env.add_environment(val_env)
            # select simulation class
            model = models[mar_env.constants['model']]
            # instantiate simulation object
            if correlations is not None:
                self.underlying_objects[asset] = model(asset, mar_env, corr=True)
            else:
                self.underlying_objects[asset] = model(asset, mar_env, corr=False)
                
        for pos in positions:
            # select right valuation class (Euro or American)
            val_class = otypes[positions[pos].otype]
            # select market env and add valuation env
            mar_env = positions[pos].mar_env
            mar_env.add_environment(self.val_env)
            # instatiate valuation class
            self.valuation_objects[pos] = val_class(name=positions[pos].name,
                                                   mar_env=mar_env,
                                                   underlying=self.underlying_objects[positions[pos].underlying],
                                                   payoff_func=positions[pos].payoff_func)
    def get_positions(self):
        '''
        Convenience method to print info about all derivatives positions in a portfolio
        '''
        for pos in self.positions:
            bar = '\n' + 50 * '-'
            print(bar)
            self.positions[pos].get_info()
            print(bar)
    def get_statistics(self):
        '''
        Provides portfolio statistics
        '''
        res_list = []
        # iterate over all positions in this portfolio
        for pos in self.positions:
            p = self.positions[pos]
            pv = value.present_value(fixed_seed=fixed_seed)
            res_list.append([
                p.name,
                p.quantity,
                # all present values for single instruments
                pv,
                value.currency,
                # single instrument times quantity
                pv * p.quantity,
                # delta of positions
                value.delta() * p.quantity,
                # vega of position
                value.vega() * p.quantity,
            ])
        # generate a DataFrame of the portfolio
        res_df = pd.DataFrame([res_list,
                              columns['name', 'quant.', 'value', 'curr.',
                                     'pos_value', 'pos_delta', 'pos_vega']])
        return res_df 